In [26]:
using Interact

In [1]:
import DrakeVisualizer

In [15]:
reload("DrakeVisualizer")

0-element Array{Any,1}

In [16]:
module Arms

import DrakeVisualizer
import DrakeVisualizer: GeometryData
using AffineTransforms
using GeometryTypes
using PyCall
import Base: convert

function __init__()
    const global spatial = pyimport("scipy.spatial")
end

function convex_hull{T}(nodes::Vector{Point{3, T}})
    hull = spatial[:ConvexHull](hcat(map(x -> convert(Vector, x), nodes)...)')
    simplices = hull[:simplices]
    simplices += 1

    faces = Face{3, Int, 0}[]
    # Reorient simplices so that normals always point out from the hull
    for i = 1:size(simplices, 1)
        v = Vector{T}[nodes[simplices[i, j]] for j in 1:size(simplices, 2)]

        # Given a face of a convex hull, all of the points in the body must be on the same side of that face. So in theory, we just need to pick one point not on the face and check which side it's on. But to attempt to be robust to numerical issues, we'll actually sum the dot product with the normal for every point in the body, and check whether that sum is positive or negative. If this becomes a performance bottleneck, we can revisit it later.
        n = Point{3, T}(cross(v[2] - v[1], v[3] - v[1]))
        b = dot(n, nodes[simplices[i, 1]])
        total = zero(T)
        for j = 1:length(nodes)
            total += dot(n, nodes[j]) - b
        end
        if total < 0
            # Then the normal is pointing the right way
            push!(faces, Face{3, Int, 0}((simplices[i,:])...))
        else
            # Otherwise the normal is pointing the wrong way, so we flip the face
            push!(faces, Face{3, Int, 0}((simplices[i,end:-1:1])...))
        end
    end
    return faces
end

type Link
    length
    surface_geometry::GeometryData
    skeleton_points::Vector{Point{3, Float64}}
end

type PlanarArm
    links::Vector{Link}
end
function two_link_arm()
    links = Array{Link}(2)
    
    link_length = 1.0
    radius = 0.1
    for i = 1:2
        surface_points = Point{3, Float64}[]
        skeleton_points = Point{3, Float64}[]
        for x = [0; link_length]
            for y = [-radius; radius]
                for z = [-radius; radius]
                    push!(surface_points, Point{3, Float64}(x, y, z))
                end
            end
        end
        surface_geometry = HomogenousMesh(surface_points, convex_hull(surface_points))
        surface_geometry_data = GeometryData(surface_geometry, tformeye(3))

        for x = linspace(0, link_length, 3)
            push!(skeleton_points, Point{3, Float64}(x, 0, 0))
        end
        links[i] = Link(link_length, surface_geometry_data, skeleton_points)
    end
    PlanarArm(links)
end

function link_origins(arm::PlanarArm, joint_angles)
    transforms = Array{AffineTransform}(length(arm.links))
    transforms[1] = tformrotate([0;0;1], joint_angles[1])
    for i = 2:length(arm.links)
        transforms[i] = transforms[i-1] * tformtranslate([arm.links[i-1].length; 0; 0]) * tformrotate([0.;0;1], joint_angles[i])
    end
    transforms
end

function convert(::Type{DrakeVisualizer.Robot}, arm::PlanarArm)
    links = [DrakeVisualizer.Link([link.surface_geometry], "link_$(i)") for (i, link) in enumerate(arm.links)]
    return DrakeVisualizer.Robot(links)
end

function skin(arm::PlanarArm)
    
    

end

Arms

In [28]:
arm = Arms.two_link_arm();
model = DrakeVisualizer.load(arm);

In [30]:
@manipulate for x1 = linspace(-pi, pi, 101), x2 = linspace(-pi, pi, 101)
    origins = Arms.link_origins(arm, [x1; x2])
    DrakeVisualizer.draw(model, origins)
end

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x1",0.0,linspace(-3.141592653589793,3.141592653589793,101),true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x2",0.0,linspace(-3.141592653589793,3.141592653589793,101),true)

nothing